In [375]:
import pandas as pd
from numpy.random import randint
#!pip install git+git://github.com/ivannz/trmf
from functools import reduce
import math
from xgboost import XGBRegressor
from sklearn.base import BaseEstimator, clone
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import ParameterGrid
from trmf import TRMFRegressor
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error as mse_score
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.base import clone
import scipy.sparse as ssp

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
#simplefilter(action='ignore', category=DeprecationWarning)

# 7. Use this file to feed into the clusterwise regression experiments


In [569]:
export = 'Export/ICDM19/'
#Train
import_ = 'import/huawei/ensemble_clusterings_ff_huawei.csv'
train = 'import/huawei/huawei_train.csv'
test = 'import/huawei/huawei_test.csv'
#import_test_yearly = 'import/M4/Yearly-test.csv'#

In [570]:
%%time
train_df = pd.read_csv(train)
clusters = pd.read_csv(import_).drop(columns ='Unnamed: 0')
test_df = pd.read_csv(test)

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 46.8 ms


In [571]:
#Let's take only with future flag = 0
test_df = test_df.loc[test_df['Unnamed: 0']==str(0.0)].drop(columns ='Unnamed: 0').fillna(0).reset_index().drop(columns = 'index')
train_df = train_df.loc[train_df['Unnamed: 0']==str(0.0)].drop(columns ='Unnamed: 0').fillna(0).reset_index().drop(columns = 'index')

In [572]:
# Add clusters
train_df.insert(1,'GMM_Pair', clusters.GMM_Pair_ff0)
train_df.insert(1,'GMM_Voting' , clusters.GMM_Voting_ff0)
train_df.insert(1,'random initialization1',randint(0,max(clusters.GMM_Pair_ff0)-1,train_df.shape[0]))
train_df.insert(1,'random initialization2',randint(0,max(clusters.GMM_Voting_ff0)-1,train_df.shape[0]))

# 8. Do hard clusterwise regression

In [573]:
base = TRMFRegressor(n_components=2,
                 n_order=2,
                 fit_regression=False,
                 fit_intercept=True,
                 nonnegative_factors=True,
                 n_max_mf_iter=5)


def gridsearch_trmf(X,test_targets):
    grid = ParameterGrid(dict(
    n_components=np.r_[1:2],#17],
    n_order=np.r_[1:2],#17],
    C_Z=np.logspace(-2, 1,num=1),#,num=4),
    C_F=np.logspace(-2, 1,num=1),#,num=4),
    C_phi=np.logspace(-2, 1,num=1),#,num=4),
    eta_Z=np.linspace(0.05, 0.95,num=1)#,num=10),
    ))
    
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = transformer.fit_transform(X)

    
    # run the experiment in parallel
    par_ = Parallel(n_jobs=-1, verbose=0)
    results = par_(delayed(helper)(par, X, test_targets.transpose(),transformer) for par in grid)
    
    keys = ['n_order', 'n_components', 'eta_Z', 'C_phi', 'C_Z', 'C_F']
    data = dict((tuple(par[k] for k in keys), rmse,) for par, rmse in results)
    sr = pd.Series(data, name="rmse").sort_index().rename_axis(keys)
    cube = sr.values.reshape(*[len(grid.param_grid[0][k]) for k in keys])
    stepping = [grid.param_grid[0][k] for k in keys]
    # find the flat index of the smallest value
    flat_index = np.argmin(cube)
    # ... and unravel into into a multidimensional index
    index = np.unravel_index(flat_index, cube.shape)
    # collect the best paramaters from the grid
    best_ = {k: grid.param_grid[0][k][i] for k, i in zip(keys, index)}
    return best_

transformer = StandardScaler(with_mean=True, with_std=True)

def helper(par, train, test,  transformer,base=base):
    basee = TRMFRegressor(n_components=1,
                     n_order=0,
                     fit_regression=False,
                     fit_intercept=True,
                     nonnegative_factors=True,
                     n_max_mf_iter=5)
    
    # clone, set parameters and fit
    trmf = clone(basee).set_params(**par).fit(train)
    
    # predict and return
    pred = transformer.inverse_transform(
        trmf.predict(n_ahead=len(test)))

    return par, mean_squared_error(test, pred)

In [574]:
def smape(y_true,y_pred,training_series= None): 
    weight = (np.abs(y_true) + np.abs(y_pred)) / 2
    output = np.divide(np.abs(y_true - y_pred), weight,
    where=weight > 0,
    out=np.full_like(weight, np.nan))
    return np.mean(np.nan_to_num(output))
    #if ((np.abs(A) + np.abs(F))==0):
   #     return 0
   # if math.isnan((100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F))))):
   #     return 0
 #   scr  = 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))
   # return np.nan_to_num(scr)

In [575]:
def pinball_loss(A,F,training_series= None,tau = 0.49):
    return np.mean(np.maximum(F - A, 0) * (1 - tau) +
                 np.maximum(F - A, 0) * tau)

In [576]:
def mape(y_true, y_pred,training_series= None): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [577]:
def MASE(testing_series, prediction_series,training_series):
    n = training_series.shape[0]
    d = np.abs(  np.diff( training_series) ).sum()/(n-1)
    errors = np.abs(testing_series - prediction_series )
    return errors.mean()/d

In [578]:
def clusterwise_regression_sklearn(X,Y,regr,regr_gridsearch_params,
                           cluster_column ='supercluster',
                            horizon = True,
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           recalculate_assignment = 100,
                           score_function = (lambda x,y: math.sqrt(mean_squared_error(x,y))),
                          ):
    """
    Runs clusterwise regression for sklearn functions
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare predictions)
    cluster_column : column with given clusters
    horizon = :  True  - 1-step horizon (1 can be modified), False - n periods ahead
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    gridsearch_limit : # of times to run GridSeach for each cluster (>=1)
    recalculate_assignment : # of assignments after which the clusters will be recalculated
    score_function : smape/ pinball_loss/ mape/(lambda x,y: math.sqrt(mean_squared_error(x,y)))
    regr : if not tmrf, then use sklearn api
    regr_gridsearch_params : if not tmrf, then use sklearn api for gridsearchCV
    ---------
    Return :
    clusters : Dictionary with trained models for each cluster
    old_cluster_for_user : Dictionary with optimal cluster for each user (after moving)
    cluster_parameters_for_gridsearch : Dictionary with optimal parameters for each cluster model
    weights
    """   
    train_targets = X.copy()
    test_targets = Y.copy()  
    n_ahead = test_targets.shape[1]
    clusters = {}
    # save cluster info for clusterwise regression
    cluster_error = {}
    cluster_has_changed = {}
    cluster_data_X = {}
    counter_for_gridsearch = 0
    old_clusters ={}
    cluster_data_true_values = {}
    cluster_predicted = {}
    score_for_user = {}
    cluster_parameters_for_gridsearch = {}
    previous_total_error = math.inf
    limit = 50
    for i in list(range(0,limit+1)): # Some limit on iterations just in case
        # Assign user to clusters
        cluster_for_user = {}
        cluster_for_user  = train_targets[cluster_column].to_dict()
        for cluster_number in train_targets[cluster_column].unique():
            # Save training data for this cluster
            
            cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
            cluster_data_true_values[cluster_number] = test_targets.loc[ cluster_data_X[cluster_number].index]
            cluster_predicted[cluster_number] =pd.DataFrame()
            if (cluster_number not in cluster_has_changed) or (cluster_has_changed[cluster_number]== True):
                    clusters[cluster_number] = clone(regr)
                    cluster_predicted[cluster_number] =pd.DataFrame()
            for slide in list(range(0,test_targets.shape[1])):
                #Train
                transformer = StandardScaler(with_mean=True, with_std=True)
                #print("X_train = ",cluster_data_X[cluster_number].iloc[:,:-1].columns)
                fitted_ = transformer.fit_transform(cluster_data_X[cluster_number].iloc[:,:-1]) # X - all col except last one
                #print("y_train = ",cluster_data_X[cluster_number].iloc[:,-1:].columns)
                clusters[cluster_number].fit(fitted_,cluster_data_X[cluster_number].iloc[:,-1:]) # y - last column
                #Test
                #delete first col
                cluster_data_X[cluster_number] = cluster_data_X[cluster_number].drop(cluster_data_X[cluster_number].columns[0], axis=1)  
                #print("X_test = ",cluster_data_X[cluster_number].columns)
                fitted_ = transformer.fit_transform(cluster_data_X[cluster_number])
                cluster_predicted[cluster_number][slide] =clusters[cluster_number].predict(fitted_).ravel() 
                #add new one at the end (what we've just predicted)
                if (horizon == False):
                    cluster_data_X[cluster_number][slide] = cluster_predicted[cluster_number][slide].values 
                else:
                    cluster_data_X[cluster_number][slide] =(cluster_data_true_values[cluster_number].iloc[:,slide])   # true values
                #print("y_true = ",cluster_data_true_values[cluster_number].columns[slide])
           # return cluster_predicted           
            # Save prediction
            cluster_predicted[cluster_number] = pd.DataFrame(cluster_predicted[cluster_number])

            cluster_predicted[cluster_number].columns  = cluster_data_true_values[cluster_number].columns 
            cluster_predicted[cluster_number].index = cluster_data_true_values[cluster_number].index
            #save cluster score
            cluster_error[cluster_number] =  score_function(cluster_data_true_values[cluster_number].values.ravel(),cluster_predicted[cluster_number].values.ravel())
        print("Error per cluster: ",cluster_error)
        # flatten true values
        all_true_values  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_data_true_values.values())  ]])   )
        # flatten prediction 
        all_predictions  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_predicted.values())  ]])   )
        total_error =  score_function(np.array(all_true_values),np.array(all_predictions))
        print("New error: ",round(total_error,4))
        # If new score is worse - load last cluster and run final prediction.
        if (total_error > previous_total_error):
            if counter_iterations ==3:
               # train_targets[cluster_column] = list(old_cluster_for_user.values())
                print('Error has increased. Backtrack to previous iteration')
                return old_clusters.copy(),old_cluster_for_user,cluster_parameters_for_gridsearch,i-counter_iterations
            counter_iterations+=1
        else:
            counter_iterations = 0 
            previous_total_error = total_error
            old_clusters = clusters.copy()
            old_cluster_for_user = cluster_for_user.copy()
       
        if (i>=limit):# return prev iteration
            return old_clusters.copy(),old_cluster_for_user,cluster_parameters_for_gridsearch,i
        # Calculate score per user
        for cluster_number in train_targets[cluster_column].unique():
            for index, row in cluster_data_true_values[cluster_number].iterrows():
                score_for_user[index] = score_function(row, cluster_predicted[cluster_number].loc[index,:])
        # Assume that we haven't change any cluster
        for x in cluster_has_changed:
            cluster_has_changed[x]=False        
        cluster_lowest_mae = min(cluster_error, key=cluster_error.get)
       # old_clusters = {}
       # old_cluster_for_user = {}
        ii =0
        reassignment_count = 0 
        if (i==0):
            old_clusters = clusters.copy()
        #Take observation x from cluster a, where error_x > error_a
        for cluster_number in train_targets[cluster_column].unique():
            # All ids in this cluster are already in cluster with lowest possible mae
            if (cluster_number == cluster_lowest_mae):
                continue
            for index, row in cluster_data_true_values[cluster_number].iterrows():    
            # User is in current cluster -> we can reassign him
                if (cluster_number  == cluster_for_user[index]):
                    if (score_for_user[index] > cluster_error[cluster_number]):
                        ii+=1#counter
                        # Have to refit changed clusters             
                        cluster_has_changed[cluster_number] = True
                        cluster_has_changed[cluster_lowest_mae] = True
                        # Move observation x to the cluster with the lowest training erro
                        train_targets.loc[index,cluster_column] = cluster_lowest_mae
                        reassignment_count+=1
                        if reassignment_count==recalculate_assignment:
                            reassignment_count=0
                            
                            # RECALCULATION - only best performing cluster (because)- TODO:save snapshot of prev cluster for backtracking
                            cluster_data_X_reassigned =  train_targets.loc[train_targets[cluster_column] == cluster_lowest_mae].drop(columns =columns_wo_TS)
                            cluster_data_true_values_reassigned= test_targets.loc[ cluster_data_X_reassigned.index]
                           
                            clusters_reassignemnt = clone(clusters[cluster_lowest_mae])
                            cluster_predicted_reassigned = pd.DataFrame()
                            for slide in list(range(0,test_targets.shape[1])):
                                #Train
                                transformer = StandardScaler(with_mean=True, with_std=True)
       
                                fitted_ = transformer.fit_transform(cluster_data_X_reassigned.iloc[:,:-1]) # X - all col except last one
                                clusters_reassignemnt.fit(fitted_,cluster_data_X_reassigned.iloc[:,-1:]) # y - last column
                                #Test
                                #delete first col
                                cluster_data_X_reassigned = cluster_data_X_reassigned.drop(cluster_data_X_reassigned.columns[0], axis=1)  
                                predd = transformer.fit_transform(cluster_data_X_reassigned)
                                cluster_predicted_reassigned[slide] =clusters[cluster_number].predict(predd).ravel() 
                                #add new one at the end (what we've just predicted)
                                if horizon == False:
                                    cluster_data_X_reassigned[slide] = cluster_predicted_reassigned[slide].values
                                else:
                                    cluster_data_X_reassigned[slide] =(cluster_data_true_values_reassigned.iloc[:,slide]) 
                            # Save prediction
                            cluster_predicted_reassigned = pd.DataFrame(cluster_predicted_reassigned)
                            cluster_error[cluster_lowest_mae] = score_function(cluster_data_true_values_reassigned.values.ravel(),cluster_predicted_reassigned.values.ravel())
                            cluster_lowest_mae = min(cluster_error, key=cluster_error.get)
        if (i==0):
            old_cluster_for_user = cluster_for_user.copy() # as well as save last assigned clusters                  
        print('Moved ', ii, ' users to another cluster \n')   
   # return old_clusters,old_cluster_for_user,cluster_parameters_for_gridsearch   

In [579]:
def final_prediction_after_clusterwise_reg_sklearn(
                           X,Y,regr,
    dataset,
    method,
    frequency,
    model,
                            horizon= True,
                           step = 'unstated',
                           clusters = None,
                           clusters_parameters = None,
                           new_clusters_for_users=None,
                           cluster_column ='supercluster',
                           scores_per_cluster = True,
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           score_function = (lambda x,y: math.sqrt(mean_squared_error(x,y))),
                           gridsearch_use = False,):
    """
    Performs final prediction and generates DataFrame for Alibaba Data
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare predictions)
    horizon : stated previously
    step : 'before_classifier' or 'after_classifier', for report generation
    clusters: dictionary with models for each cluster. If None, then refit
    clusters_parameters: dictionary with parameters for each cluster model
    new_clusters_for_users: dictionary with clusters for each user. 
    cluster_column : column with given clusters
    
    scores_per_cluster : if False the score will be given for the all dataset, not by cluster
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    score_function : smape/ pinball_loss/ mape/(lambda x,y: math.sqrt(mean_squared_error(x,y)))
    gridsearch_use : Run gridSeach (True) or use clusters_parameters (False)?
    regr : if not tmrf, then use sklearn api
    regr_gridsearch_params : if not tmrf, then use sklearn api for gridsearchCV
    ---------
    Return :
    result : DataFrame with columns
    cluster, user id, true target, prediction, rmse, model, method (all data, clustered), step (all data, before classifier, after classifier)
    """
    result = pd.DataFrame()
    train_targets = X.copy()
    test_targets = Y.copy()
    n_ahead = test_targets.shape[1]
    if (new_clusters_for_users is not None):
        # we assume that the order in DataFrame is the same as in clusterwise regression
        train_targets[cluster_column]=list(cluster_per_user.values()) 
    cluster_data_X = {}
    cluster_data_true_values = {}
    if (clusters_parameters == None or gridsearch_use==True):
        clusters_parameters = {}
    if (clusters == None or clusters_parameters ==None):
        clusters = {}
    for cluster_number in train_targets[cluster_column].unique():
        cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
        cluster_data_true_values[cluster_number] = test_targets.iloc[ cluster_data_X[cluster_number].index]
        cluster_data_true_values[cluster_number].index =   test_targets.iloc[ cluster_data_X[cluster_number].index].index
        predicted =pd.DataFrame()
        
        if (cluster_number not in clusters or ( cluster_number not in  clusters_parameters) or gridsearch_use==True):
            clusters[cluster_number] = clone(regr)
            
        for slide in list(range(0,test_targets.shape[1])):
            #Train
            transformer = StandardScaler(with_mean=True, with_std=True)
            #print("X_train = ",cluster_data_X[cluster_number].iloc[:,:-1].columns)
            fitted_ = transformer.fit_transform(cluster_data_X[cluster_number].iloc[:,:-1]) # X - all col except last one
            #print("y_train = ",cluster_data_X[cluster_number].iloc[:,-1:].columns)
           # print(cluster_data_X[cluster_number].iloc[:,-1:])
            clusters[cluster_number].fit(fitted_,cluster_data_X[cluster_number].iloc[:,-1:]) # y - last column
            #Test
            #delete first col
            cluster_data_X[cluster_number] = cluster_data_X[cluster_number].drop(cluster_data_X[cluster_number].columns[0], axis=1)  
            #print("X_test = ",cluster_data_X[cluster_number].columns)
            fitted_ = transformer.fit_transform(cluster_data_X[cluster_number])
            predicted[slide] =clusters[cluster_number].predict(fitted_).ravel() 
            #add new one at the end (what we've just predicted)
            if (horizon == False):
                cluster_data_X[cluster_number][slide] = predicted[slide].values 
            else:
              #  print(cluster_data_true_values[cluster_number].iloc[:,slide])
                cluster_data_X[cluster_number][slide] =(cluster_data_true_values[cluster_number].iloc[:,slide]).values   # true values
        
        
        df_prediction = pd.DataFrame(predicted)
        df_prediction.columns  = cluster_data_true_values[cluster_number].columns 
        df_prediction.index = cluster_data_true_values[cluster_number].index
       # df_prediction.index =  train_targets.loc[train_targets[cluster_column] == cluster_number]["Unnamed: 1"]
        # Transform prediction for output
        #MODEL        
        
        df_prediction.insert(0,'Score',score_function(cluster_data_true_values[cluster_number].values.ravel(),df_prediction.values.ravel(),training_series=fitted_))
        df_prediction.insert(0,'model',model)
        #Dataset
        df_prediction.insert(0,'dataset',dataset)
        #METHOD
        df_prediction.insert(0,'method',method)
        #STEP
        df_prediction.insert(0,'frequency',frequency)
        if horizon == True:
            df_prediction.insert(0,'horizon',str(1)+'_step')
        else:
            df_prediction.insert(0,'horizon','full')
        # CLUSTER
        df_prediction.insert(0,'cluster_id',str(cluster_column)+'_'+str(cluster_number))
        #Wide to logn
        #print(pd.DataFrame(df_prediction))
        df_prediction = df_prediction.reset_index().melt(id_vars =['Unnamed: 1','cluster_id','model','horizon','dataset','method','frequency','Score']).rename(index = str, columns={'variable':'date','value':'predicted_value','Unnamed: 1':'id'})
        true_values=cluster_data_true_values[cluster_number].reset_index().melt(id_vars=['Unnamed: 1']).rename(index = str, columns={'variable':'date','value':'true_value','Unnamed: 1':'id'})
        result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))
    if scores_per_cluster == False:
           result.Score = score_function(result.true_value,result.predicted_value,
                             training_series=train_targets.drop(columns =columns_wo_TS).values.ravel())
    return result   

In [580]:
def clusterwise_regression(X,Y,
                           cluster_column ='supercluster',
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           gridsearch_limit = 1,
                           recalculate_assignment = 100,
                           score_function = (lambda x,y: math.sqrt(mean_squared_error(x,y))),
                          ):
    """
    Runs clusterwise regression
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare predictions)
    cluster_column : column with given clusters
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    gridsearch_limit : # of times to run GridSeach for each cluster (>=1)
    recalculate_assignment : # of assignments after which the clusters will be recalculated
    score_function : smape/ pinball_loss/ mape/(lambda x,y: math.sqrt(mean_squared_error(x,y)))
    regr : if not tmrf, then use sklearn api
    regr_gridsearch_params : if not tmrf, then use sklearn api for gridsearchCV
    ---------
    Return :
    clusters : Dictionary with trained models for each cluster
    old_cluster_for_user : Dictionary with optimal cluster for each user (after moving)
    cluster_parameters_for_gridsearch : Dictionary with optimal parameters for each cluster model
    weights
    """
        
        
    train_targets = X.copy()
    test_targets = Y.copy()  # test_Y 
    n_ahead = test_targets.shape[1]
    clusters = {}
    # save cluster info for clusterwise regression
    cluster_error = {}
    cluster_has_changed = {}
    cluster_data_X = {}
    counter_for_gridsearch = 0
    old_clusters ={}
    counter_iterations = 0 
    cluster_data_true_values = {}
    cluster_predicted = {}
    score_for_user = {}
    cluster_parameters_for_gridsearch = {}
    previous_total_error = math.inf
    limit = 50
    for i in list(range(0,limit+1)): # Some limit on iterations just in case
        # Assign user to clusters
        cluster_for_user = {}
        cluster_for_user  = train_targets[cluster_column].to_dict()
        for cluster_number in train_targets[cluster_column].unique():
            cluster_predicted[cluster_number] =pd.DataFrame()
            # Save training data for this cluster
            cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
            cluster_data_true_values[cluster_number] = test_targets.loc[ cluster_data_X[cluster_number].index]
            # Scaling (Not minmax because of it wasn't used in trmf github)
            transformer = StandardScaler(with_mean=True, with_std=True)
            cluster_data_X[cluster_number] = cluster_data_X[cluster_number].transpose()
           # print(cluster_number)
           # print(cluster_data_X[cluster_number])
            cluster_data_X[cluster_number] = transformer.fit_transform(cluster_data_X[cluster_number])
            #GridSearchCV for TRMF
            # If initial initialization or cluster has changed
            if (cluster_number not in cluster_has_changed) or (cluster_has_changed[cluster_number]== True):
                if (i<gridsearch_limit):
                    # add prev_cluster_paraeters
                    cluster_parameters_for_gridsearch[cluster_number] = gridsearch_trmf(cluster_data_X[cluster_number],cluster_data_true_values[cluster_number])
                clusters[cluster_number ]= TRMFRegressor(**cluster_parameters_for_gridsearch[cluster_number], eta_F=0., adj=None, C_B=0., fit_regression=False,
                                 fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                                 z_step_kind="tron")
                clusters[cluster_number].fit(cluster_data_X[cluster_number])
            fitted_ = np.dot(clusters[cluster_number].factors_,clusters[cluster_number].loadings_)+clusters[cluster_number].intercept_
            cluster_predicted[cluster_number] = transformer.inverse_transform( clusters[cluster_number].predict(fitted_,n_ahead))
            
            # Save prediction
            cluster_predicted[cluster_number] = pd.DataFrame(cluster_predicted[cluster_number]).transpose()
            
           # print(cluster_predicted[cluster_number].columns )
           # print( cluster_data_true_values[cluster_number].columns)
            cluster_predicted[cluster_number].columns  = cluster_data_true_values[cluster_number].columns 
            cluster_predicted[cluster_number].index = cluster_data_true_values[cluster_number].index
            #save cluster score
            cluster_error[cluster_number] =  score_function(cluster_data_true_values[cluster_number].values.ravel(),cluster_predicted[cluster_number].values.ravel())
        print("Error per cluster: ",cluster_error)
        # flatten true values
        all_true_values  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_data_true_values.values())  ]])   )
        # flatten prediction 
        all_predictions  = reduce(lambda x,y: list(x)+list(y), ([x.ravel() for x in [x.values for x in list(cluster_predicted.values())  ]])   )
        total_error =  score_function(np.array(all_true_values),np.array(all_predictions))
        print("New error: ",round(total_error,4))
        # If new score is worse - load last cluster and run final prediction. 
        if (total_error > previous_total_error):
            if counter_iterations ==3:
               # train_targets[cluster_column] = list(old_cluster_for_user.values())
                print('Error has increased. Backtrack to previous iteration')
                return old_clusters.copy(),old_cluster_for_user,cluster_parameters_for_gridsearch,i-counter_iterations
            counter_iterations+=1
        else:
            counter_iterations = 0 
            previous_total_error = total_error
            old_clusters = clusters.copy()
            old_cluster_for_user = cluster_for_user.copy()
            # Save all stuff
        if (i>=limit):# return prev iteration
            return old_clusters.copy(),old_cluster_for_user,cluster_parameters_for_gridsearch,i
        # Calculate score per user
        for cluster_number in train_targets[cluster_column].unique():
            for index, row in cluster_data_true_values[cluster_number].iterrows():
                score_for_user[index] = score_function(row, cluster_predicted[cluster_number].loc[index,:])
        # Assume that we haven't change any cluster
        for x in cluster_has_changed:
            cluster_has_changed[x]=False        
        cluster_lowest_mae = min(cluster_error, key=cluster_error.get)
       # old_clusters = {}
        #old_cluster_for_user = {}
        ii =0
        reassignment_count = 0
        if (i==0):
            old_clusters = clusters.copy()
        #Take observation x from cluster a, where error_x > error_a
        for cluster_number in train_targets[cluster_column].unique():
            # All ids in this cluster are already in cluster with lowest possible mae
            if (cluster_number == cluster_lowest_mae):
                continue
            for index, row in cluster_data_true_values[cluster_number].iterrows():    
            # User is in current cluster -> we can reassign him
                if (cluster_number  == cluster_for_user[index]):
                    if (score_for_user[index] > cluster_error[cluster_number]):
                        ii+=1#counter
                        # Have to refit changed clusters             
                        cluster_has_changed[cluster_number] = True
                        cluster_has_changed[cluster_lowest_mae] = True
                        # Move observation x to the cluster with the lowest training erro
                        train_targets.loc[index,cluster_column] = cluster_lowest_mae
                        reassignment_count+=1
                        if reassignment_count==recalculate_assignment:
                            reassignment_count=0
                            # RECALCULATION - only best performing cluster (because)- TODO:save snapshot of prev cluster for backtracking
                            cluster_data_X_reassigned =  train_targets.loc[train_targets[cluster_column] == cluster_lowest_mae].drop(columns =columns_wo_TS)
                            
                            # print(  train_targets.loc[train_targets.cluster==2].drop(columns = ['cluster','V1']).shape)
                            cluster_data_true_values_reassigned= test_targets.loc[ cluster_data_X_reassigned.index]
                            cluster_data_X_reassigned =  transformer.fit_transform(cluster_data_X_reassigned.transpose())
                            clusters_reassignemnt = clone(clusters[cluster_lowest_mae])
                            clusters_reassignemnt.fit(cluster_data_X_reassigned)
                            predd =  transformer.inverse_transform(clusters_reassignemnt.predict(cluster_data_X_reassigned ,n_ahead=n_ahead))
                            cluster_predicted_reassigned = predd
                            # Save prediction
                            cluster_predicted_reassigned = pd.DataFrame(cluster_predicted_reassigned).transpose()
                            cluster_error[cluster_lowest_mae] = score_function(cluster_data_true_values_reassigned.values.ravel(),cluster_predicted_reassigned.values.ravel())
                            cluster_lowest_mae = min(cluster_error, key=cluster_error.get)
        if (i==0):
            old_cluster_for_user = cluster_for_user.copy() # as well as save last assigned clusters                  
        print('Moved ', ii, ' users to another cluster \n')   
   # return old_clusters,old_cluster_for_user,cluster_parameters_for_gridsearch   

In [581]:
def final_prediction_after_clusterwise_reg(
    X,Y,
        dataset,
    method,
    frequency,
    model,           
                           step = 'unstated',
                           clusters = None,
                           clusters_parameters = None,
                           new_clusters_for_users=None,
                           cluster_column ='supercluster',
                           scores_per_cluster = True,
                           columns_wo_TS = ['clusters_r','clusters_m','clusters_f','supercluster'],
                           score_function = (lambda x,y: math.sqrt(mean_squared_error(x,y))),
                           gridsearch_use = False):
    """
    Performs final prediction and generates DataFrame for Alibaba Data
    Parameters
    ----------
    X : DataFrame with train data (rows: id; columns: clusters + TS)
    Y : DataFrame with true_values (rows: id; columns: clusters + TS) (to compare predictions)
    step : 'before_classifier' or 'after_classifier', for report generation
    clusters: dictionary with models for each cluster. If None, then refit
    clusters_parameters: dictionary with parameters for each cluster model
    new_clusters_for_users: dictionary with clusters for each user. 
    cluster_column : column with given clusters
    
    scores_per_cluster : if False the score will be given for the all dataset, not by cluster
    columns_wo_TS  : columns in DataFrames not regarded as TimeSeries
    score_function : smape/ pinball_loss/ mape/(lambda x,y: math.sqrt(mean_squared_error(x,y)))
    gridsearch_use : Run gridSeach (True) or use clusters_parameters (False)?
    regr : if not tmrf, then use sklearn api
    regr_gridsearch_params : if not tmrf, then use sklearn api for gridsearchCV
    ---------
    Return :
    result : DataFrame with columns
    cluster, user id, true target, prediction, rmse, model, method (all data, clustered), step (all data, before classifier, after classifier)
    """
    result = pd.DataFrame()
    train_targets = X.copy()
    test_targets = Y.copy()
    n_ahead = test_targets.shape[1]
    if (new_clusters_for_users is not None):
        # we assume that the order in DataFrame is the same as in clusterwise regression
        train_targets[cluster_column]=list(cluster_per_user.values()) 
    cluster_data_X = {}
    cluster_data_true_values = {}
    if (clusters_parameters == None or gridsearch_use==True):
        clusters_parameters = {}
    if (clusters == None or clusters_parameters ==None):
        clusters = {}
    for cluster_number in train_targets[cluster_column].unique():
        cluster_data_X[cluster_number] = train_targets.loc[train_targets[cluster_column] == cluster_number].drop(columns =columns_wo_TS)
        cluster_data_true_values[cluster_number] = test_targets.iloc[ cluster_data_X[cluster_number].index]
        cluster_data_true_values[cluster_number].index =   test_targets.iloc[ cluster_data_X[cluster_number].index].index
        # Scaling (Not minmax because of it wasn't used in trmf github)
        transformer = StandardScaler(with_mean=True, with_std=True)
        
        cluster_data_X[cluster_number] = cluster_data_X[cluster_number].transpose()
        cluster_data_X[cluster_number] = transformer.fit_transform(cluster_data_X[cluster_number])
        if (cluster_number not in clusters_parameters or gridsearch_use==True):
            clusters_parameters[cluster_number] = gridsearch_trmf(cluster_data_X[cluster_number],cluster_data_true_values[cluster_number])
                    
        if (cluster_number not in clusters or ( cluster_number not in  clusters_parameters) or gridsearch_use==True):
            clusters[cluster_number ]= TRMFRegressor(**clusters_parameters[cluster_number], eta_F=0., adj=None, C_B=0., fit_regression=False,
                             fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                             z_step_kind="tron")
                # If you feed clusters from clusterwise, you don't need to refit them
            clusters[cluster_number ].fit(cluster_data_X[cluster_number])
        fitted_ = np.dot(clusters[cluster_number].factors_,clusters[cluster_number].loadings_)+clusters[cluster_number].intercept_
        predicted = transformer.inverse_transform( clusters[cluster_number].predict(fitted_,n_ahead))

            
        df_prediction = pd.DataFrame(predicted).transpose()
        df_prediction.columns  = cluster_data_true_values[cluster_number].columns 
        df_prediction.index = cluster_data_true_values[cluster_number].index # here we need to set old cluster
    
        # Transform prediction for output
        #MODEL
        df_prediction.insert(0,'Score',score_function(cluster_data_true_values[cluster_number].values.ravel(),df_prediction.values.ravel(),training_series=fitted_))
        df_prediction.insert(0,'model',model)
        #Dataset
        df_prediction.insert(0,'dataset',dataset)
        #METHOD
        df_prediction.insert(0,'method',method)
        #STEP
        df_prediction.insert(0,'frequency',frequency)
        df_prediction.insert(0,'horizon','full')
        df_prediction.insert(0,'cluster_id',str(cluster_column)+'_'+str(cluster_number))
        #Wide to logn
        df_prediction = df_prediction.reset_index().melt(id_vars =['Unnamed: 1','cluster_id','model','horizon','dataset','method','frequency','Score']).rename(index = str, columns={'variable':'date','value':'predicted_value','Unnamed: 1':'id'})
        true_values=cluster_data_true_values[cluster_number].reset_index().melt(id_vars=['Unnamed: 1']).rename(index = str, columns={'variable':'date','value':'true_value','Unnamed: 1':'id'})
        result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))
    if scores_per_cluster == False:
           result.Score = score_function(result.true_value,result.predicted_value,
                             training_series=train_targets.drop(columns =columns_wo_TS).values.ravel())
    return result   

In [700]:
# TODO: Check if it can work with Gridsearch CV(It should but, you can just pass None)
class RegressorEnsemble(BaseEstimator):
    def __init__(self, rgr, n_estimators=10):
        self.rgr = rgr
        self.n_estimators = n_estimators
        self.rgrs = []
        for i in range(self.n_estimators):
            self.rgrs.append(clone(self.rgr))

    def fit(self, X, y,
              seed=None,):
        if seed is not None:
            np.random.seed(seed)
        sd = np.random.seed()
        print(sd)
        for i in range(self.n_estimators):
            self.rgrs[i].fit(X, y,random_seed = sd)

    def predict(self, X):
        ans = self.rgrs[0].predict(X)
        for i in range(1,self.n_estimators):
            ans += self.rgrs[i].predict(X)
        return ans / len(self.rgrs)

In [583]:
regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,max_depth=3,
                                                          random_state=0))
regr_multirf = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror',
                                                     n_estimators=50)) #tree_method='gpu_hist'))

# Reporting

In [671]:
res=pd.DataFrame()

## TRMF

In [681]:
%%time
# FOR TRMF
freq = 'future_flag_0'
col = 'random initialization2'
X = train_df
Y  = test_df
clusters, cluster_per_user,parameters,reassignment_iterations = clusterwise_regression(X,#.drop(columns =['V'+str(x) for x in list(range(2,817))]),
                                                                    Y.drop(columns = ['Unnamed: 1']),
                                                                    cluster_column= col,
                columns_wo_TS=['Unnamed: 1','GMM_Pair','GMM_Voting','random initialization1','random initialization2'],
                                                                        score_function=smape,
                                                                       recalculate_assignment=10000)

Error per cluster:  {1: 1.655143293378799, 2: 1.7406956276444272, 5: 1.6956489267182913, 3: 1.5392566880983616, 4: 1.6822268625650296, 6: 1.6630462243516302, 0: 1.583777351492016}
New error:  1.6565
Moved  500  users to another cluster 

Error per cluster:  {1: 1.0344274390689168, 2: 1.2879851643512459, 5: 1.058037744880844, 3: 1.8796138836006555, 4: 1.1130547791236602, 6: 1.1192378024890657, 0: 1.0022843691531855}
New error:  1.6538
Moved  563  users to another cluster 

Error per cluster:  {1: 0.7802264546030984, 2: 0.9253668941161803, 5: 0.8093429039916029, 3: 1.5502803486558048, 4: 0.9057622691565554, 6: 0.7750852323252116, 0: 1.8280205251929875}
New error:  1.6643
Moved  592  users to another cluster 

Error per cluster:  {1: 0.5952810997332433, 2: 0.7446109094576104, 5: 0.6267769135407741, 3: 0.9522093853939413, 4: 0.8086805384665287, 6: 1.8343630618563296, 0: 1.3047581545444658}
New error:  1.6487
Moved  607  users to another cluster 

Error per cluster:  {1: 1.828417290979273, 

In [682]:
dff_trmf = final_prediction_after_clusterwise_reg(X,
                                    Y.set_index(['Unnamed: 1']),
                                    step ='before_classifier',
                                    clusters = clusters,
                                    clusters_parameters = parameters,
                                    new_clusters_for_users = cluster_per_user,
                                    gridsearch_use = None,
                                    cluster_column=col,
                                    scores_per_cluster = False , #important !False,
                                    score_function=smape,#smape,#pinball_loss,#  (lambda x,y: math.sqrt(mean_squared_error(x,y)))
                                    columns_wo_TS=['Unnamed: 1','GMM_Pair','GMM_Voting','random initialization1','random initialization2'],
                                    
                                    dataset = 'Huawei',
                                    model = 'TRMF',
                                    method = 'clusterwise',
                                    frequency = freq)

In [683]:
dff_trmf['clusterwise_loss'] = 'SMAPE'
dff_trmf['final_loss'] = 'SMAPE'
dff_trmf['reassignments_iterations'] = reassignment_iterations
res = res.append(dff_trmf)

## SKLEARN 

In [695]:
%%time
# FOR SKLEARN
X = train_df
Y  = test_df
modl = 'XGBoost'
#regr = Ridge()
regr = RegressorEnsemble(XGBRegressor(objective='reg:squarederror',n_estimators=125),1)
col = 'random initialization2'
freq = 'future_flag_0'
clusters, cluster_per_user,parameters,reassignment_iterations = clusterwise_regression_sklearn(X,#.drop(columns =['V'+str(x) for x in list(range(2,817))]),
                                                                    Y.drop(columns = ['Unnamed: 1']),
                                                                    cluster_column=col,
                                                                        columns_wo_TS=['Unnamed: 1','GMM_Pair','GMM_Voting','random initialization1','random initialization2'],
                                                                        score_function=smape,
                                                                       recalculate_assignment=1000,
                                                                      horizon = True,
                                                                       regr =regr,
                                                                      regr_gridsearch_params = None
                                                                       )

Error per cluster:  {1: 1.7148025965740117, 2: 1.7510526493997172, 5: 1.730354251999149, 3: 1.6354576429519745, 4: 1.7405710417553748, 6: 1.7526378752191831, 0: 1.6233113503045031}
New error:  1.7119
Moved  494  users to another cluster 

Error per cluster:  {1: 1.2255791334524961, 2: 1.259721629337031, 5: 1.2217348154843444, 3: 1.1378892613955842, 4: 1.3307294868693147, 6: 1.3097996396594433, 0: 1.9018749309383731}
New error:  1.7176
Moved  549  users to another cluster 

Error per cluster:  {1: 0.9223999123564584, 2: 0.9059144655091345, 5: 0.9441557746745175, 3: 1.8475539262203169, 4: 1.0065735945742702, 6: 1.010761453475965, 0: 1.6526415413169664}
New error:  1.7085
Moved  587  users to another cluster 

Error per cluster:  {1: 0.889937850636588, 2: 1.8735176945168575, 5: 0.7908028024041374, 3: 1.4989893725060004, 4: 0.8021920921931126, 6: 0.9012400449949423, 0: 1.1943529589828135}
New error:  1.7158
Moved  581  users to another cluster 

Error per cluster:  {1: 0.8018835394535708, 

In [696]:
dff_sklearn = final_prediction_after_clusterwise_reg_sklearn(X,
                                    Y.set_index(['Unnamed: 1']),
                                    regr = regr,
                                    clusters = clusters,
                                    clusters_parameters = parameters,
                                    new_clusters_for_users = cluster_per_user,
                                    gridsearch_use = None,
                                    cluster_column=col,
                                    horizon = True, 
                                                            

                                    scores_per_cluster = False , #important !False,
                                    score_function=smape,#smape,#pinball_loss,#  (lambda x,y: math.sqrt(mean_squared_error(x,y)))
                                    columns_wo_TS=['Unnamed: 1','GMM_Pair','GMM_Voting','random initialization1','random initialization2'],

                                    dataset = 'Huawei',
                                    model = modl,
                                    method = 'clusterwise',
                                    frequency = freq)

In [697]:
dff_sklearn['clusterwise_loss'] = 'SMAPE'
dff_sklearn['final_loss'] = 'SMAPE'
dff_sklearn['reassignments_iterations']  =reassignment_iterations
res = res.append(dff_sklearn)

## Saving file

In [698]:
import pandas as pd
import time


def write_df(df: pd.DataFrame,
             path: str,
             project: str,
             step: str,
             format: str = 'csv') -> None:


    title = path + time.strftime("%Y%m%d_%H%M") + '_' + project + '__' + step + '.' + format

    if format == 'csv':
        df.to_csv(title)

    if format == 'h5':
        df.to_hdf(title, step, table=True, mode='a')

In [699]:
write_df(res, export,'huawei','1')

In [669]:
#pd.read_csv("Export/ICDM19/20190605_2237_huawei__1.csv").reassignments_iterations.
#unique()

array([ 2,  3,  7, 12, 10])